<a href="https://colab.research.google.com/github/Valentina-Fontanarosa/wellness-forecast-rs/blob/main/Modello_previsione_benessere_giornaliero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install numpy pandas scipy matplotlib seaborn bokeh statsmodels scikit-learn utilsforecast

In [12]:
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# Hide warnings
# ==============================================================================
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

# Handling and processing of Data
# ==============================================================================
import numpy as np
import pandas as pd

# Handling and processing of Data for Date (time)
# ==============================================================================
import datetime

# Plot
# ==============================================================================
import matplotlib.pyplot as plt

# Modelling with Sklearn
# ==============================================================================
from sklearn.linear_model import LinearRegression

# Define the plot size
# ==============================================================================
from pylab import rcParams
rcParams['figure.figsize'] = (18,7)


# **Caso di studio: previsione del benessere giornaliero di un utente**

L’obiettivo di questo progetto è valutare se, attraverso modelli di deep learning per il forecasting, sia possibile prevedere il livello di benessere giornaliero di un utente, rappresentato da un indice sintetico chiamato ***activity index*** , utilizzando le informazioni relative alla durata del sonno e all’andamento dell’attività fisica, come la velocità del passo.

# **Dataset originale**

Il dataset originale è composto da 4 colonne:

* `user_id`: identificatore dell'utente
* `date`: giorno in cui è stato raccolto il dato aggregato
* `data_type`: intero che identifica la tipologia di dato raccolto
* `data_value`: valore associato al dato raccolto

I diversi valori della variabile `data_type` si riferiscono a dati aggregati raccolti giornalmente dal dispositivo di ogni singolo utente e identificano le tipologie di dato in base al seguente schema:

1.  numero totale di passi effettuati
2.  peso (kg)
3.  BMI (kg/m^2)
4.  pressione sanguigna sistolica (mmHg)
5.  velocità dell'onda sfigmica arteriosa (PWV), (m/s)
6.  PWV healthiness (1: bassa, 2: sano, 3: troppo alta)
7.  frequenza cardiaca media (bpm)
8.  frequenza cardiaca minima (bpm)
9.  frequenza cardiaca massima (bpm)
10. durata del sonno (ore)
11. orario in cui l'utente si è messo a letto
12. orario in cui l'utente si è alzato dal letto
13. numero di volte in cui l'utente si è svegliato durante il sonno
14. durata del tempo in cui l'utente si è svegliato durante il sonno (ore)
15. tempo impiegato dall'utente per addormentarsi (ore)
16. tempo impiegato dall'utente per alzarsi dal letto (ore)
17. durata di sonno leggero (ore)
18. durata di sonno REM (ore)
19. durata di sonno profondo (ore)
20. tipo di attività
21. durata dell'attività (secondi)
22. calorie consumate durante l'attività
23. frequenza cardiaca media durante l'attività (bpm)
24. frequenza cardiaca minima durante l'attività (bpm)
25. frequenza cardiaca massima durante l'attività (bpm)
26. velocità dell'andatura dei passi (passi al minuto)
27. velocità dell'andatura a distanza (km all'ora)

In [13]:
from google.colab import drive
drive.mount('/content/drive')
MYDRIVE = 'drive/MyDrive/Datasets/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
df = pd.read_csv(MYDRIVE + "data/data.csv", usecols=[1, 2, 3, 4])
df.columns = ["user_id", "date_local", "data_type", "value"]
print(df.head())

                                    user_id  date_local  data_type     value
0  f3ca4233a8b87472465aa3e0bddc9cc32dd27152  2016-04-01          1  10037.00
1  f3ca4233a8b87472465aa3e0bddc9cc32dd27152  2016-04-01          2     86.51
2  f3ca4233a8b87472465aa3e0bddc9cc32dd27152  2016-04-01          3     28.91
3  f3ca4233a8b87472465aa3e0bddc9cc32dd27152  2016-04-01          7    117.00
4  f3ca4233a8b87472465aa3e0bddc9cc32dd27152  2016-04-01          8    117.00


# **Creazione dataset per la previsione**

E' stato costruito un dataset personalizzato contenente esclusivamente i dati utili all’analisi, con particolare focus sulle variabili legate al sonno e all’attività fisica giornaliera di ciascun utente, in particolare sono state estratte le feature: *sleepduration, bedin, bedout, nbawake, awakeduration, timetosleep, timetowake, remduration, deepduration, activity_type, activity_duration, activity_calories, stepsgaitspeed, distancegaitspeed*.

In [15]:
# Mappatura data_type → nome feature
feature_map = {
    10: "sleepduration", 11: "bedin", 12: "bedout", 13: "nbawake",
    14: "awakeduration", 15: "timetosleep", 16: "timetowake", 18: "remduration",
    19: "deepduration", 20: "activity_type", 21: "activity_duration",
    22: "activity_calories", 26: "stepsgaitspeed", 27: "distancegaitspeed"
}

# Creazione lista di DataFrame per ciascuna feature
dataset_list = [
    df[df['data_type'] == dtype][['user_id', 'date_local', 'value']]
    .rename(columns={'date_local': 'date', 'value': name})
    for dtype, name in feature_map.items()
]

# Merge progressivo su 'user_id' e 'date'
from functools import reduce

dataset = reduce(lambda left, right: pd.merge(left, right, on=['user_id', 'date'], how='outer'), dataset_list)

# Pulizia finale
dataset.dropna(subset=['sleepduration', 'stepsgaitspeed'], inplace=True)
dataset.set_index(['user_id', 'date'], inplace=True)

# Salvataggio
dataset.to_csv(MYDRIVE + 'data/processed_dataset.csv')

In [16]:
dataset = pd.read_csv(MYDRIVE + 'data/processed_dataset.csv', index_col=[0, 1])  # Assicura che user_id e date siano indici
dataset

sleepduration  bedin  \
user_id                                  date                               
0000a344b7eb90edec5801f1562e547b3d3bf1c7 2016-08-08           8.72  22.10   
                                         2016-08-10           8.35  21.90   
                                         2016-08-11           7.15  23.07   
                                         2016-08-12           7.10  22.93   
                                         2016-08-13          10.08  22.90   
...                                                            ...    ...   
fffcad1ee384a6299d314983e5b8bf2fa2d4f41c 2017-03-20           7.87  22.43   
                                         2017-03-21           7.35  23.20   
                                         2017-03-23           5.78  23.58   
                                         2017-03-24           7.25  23.43   
                                         2017-03-28           6.70  23.90   

                                                     bedout  nbawake  \
user_id                                  date                          
0000a344b7eb90edec5801f1562e547b3d3bf1c7 2016-08-08    7.30      3.0   
                                         2016-08-10    7.62      3.0   
                                         2016-08-11    6.83      5.0   
                                         2016-08-12    6.72      4.0   
                                         2016-08-13    9.75      4.0   
...                                                     ...      ...   
fffcad1ee384a6299d314983e5b8bf2fa2d4f41c 2017-03-20    7.03      2.0   
                                         2017-03-21    7.08      2.0   
                                         2017-03-23    5.82      3.0   
                                         2017-03-24    7.17      3.0   
                                         2017-03-28    7.32      3.0   

                                                     awakeduration  \
user_id                                  date                        
0000a344b7eb90edec5801f1562e547b3d3bf1c7 2016-08-08           0.48   
                                         2016-08-10           0.48   
                                         2016-08-11           0.62   
                                         2016-08-12           0.68   
                                         2016-08-13           0.77   
...                                                            ...   
fffcad1ee384a6299d314983e5b8bf2fa2d4f41c 2017-03-20           0.70   
                                         2017-03-21           0.53   
                                         2017-03-23           0.40   
                                         2017-03-24           0.48   
                                         2017-03-28           0.72   

                                                     timetosleep  timetowake  \
user_id                                  date                                  
0000a344b7eb90edec5801f1562e547b3d3bf1c7 2016-08-08         0.10         0.0   
                                         2016-08-10         0.00         0.0   
                                         2016-08-11         0.00         0.0   
                                         2016-08-12         0.05         0.0   
                                         2016-08-13         0.30         0.0   
...                                                          ...         ...   
fffcad1ee384a6299d314983e5b8bf2fa2d4f41c 2017-03-20         0.03         0.0   
                                         2017-03-21         0.00         0.0   
                                         2017-03-23         0.05         0.0   
                                         2017-03-24         0.00         0.0   
                                         2017-03-28         0.00         0.0   

                                                     remduration  \
user_id                                  date                      
0000a344b7eb90edec5801f156

# **Calcolo indice di attività fisica**

La funzione calcola un punteggio (activity_index) per ogni riga del dataset (cioè per ogni giorno di ogni utente), bilanciando:

- Bonus per attività e qualità del sonno, ad esempio chi fa movimento e consuma calorie
- Penalizza poco movimento, poco sonno e risvegli notturni

Questo indice, ottenuto tramite una funzione appositamente definita, combina diverse informazioni relative all’attività fisica e alla qualità del sonno (ad esempio calorie bruciate, durata del sonno e risvegli notturni) in un unico valore numerico utile per la previsione del benessere giornaliero.

In [17]:
def get_activity_score(row):
    score = 0

    # Calorie bruciate
    if not pd.isna(row['activity_calories']):
        if row['activity_calories'] > 500:
            score += 3
        elif row['activity_calories'] > 300:
            score += 2
        elif row['activity_calories'] > 100:
            score += 1

    # Durata attività fisica
    if not pd.isna(row['activity_duration']):
        duration_min = row['activity_duration'] / 60
        if duration_min > 60:
            score += 3
        elif duration_min > 30:
            score += 2
        elif duration_min > 10:
            score += 1

    # Sleepduration
    if not pd.isna(row['sleepduration']):
        if row['sleepduration'] >= 7:
            score += 2
        elif row['sleepduration'] >= 6:
            score += 1

    # Awakeduration
    if not pd.isna(row['awakeduration']):
        if row['awakeduration'] <= 0.5:
            score += 2
        elif row['awakeduration'] <= 1:
            score += 1

    # Deep sleep
    if not pd.isna(row['deepduration']):
        if row['deepduration'] > 1:
            score += 1

    # REM
    if not pd.isna(row['remduration']):
        if 1 <= row['remduration'] <= 2:
            score += 1

    # Passo veloce
    if not pd.isna(row['stepsgaitspeed']):
        if row['stepsgaitspeed'] > 100:
            score += 1

    # Distanza
    if not pd.isna(row['distancegaitspeed']):
        if row['distancegaitspeed'] > 2:
            score += 1

    return score


La colonna activity_index viene calcolata e aggiunta al dataset.

In [18]:
dataset['activity_index'] = dataset.apply(get_activity_score, axis=1)


# **Creazione sequenze valide di 15 giorni**

Per la preparazione dei dati è stato implementato un algoritmo che costruisce sequenze temporali di 15 giorni consecutivi (con massimo un giorno di gap) per ciascun utente. Questo passaggio consente di trasformare i dati grezzi in finestre temporali strutturate, fondamentali per permettere al modello Informer di apprendere le dinamiche temporali e fare previsioni sull'andamento futuro dell'activity index, stimando il valore dell’indice di attività del giorno successivo alla sequenza, ovvero il sedicesimo giorno.

In [19]:
target_sequence_len = 15
sequences = []
target = []

# Resettiamo e convertiamo 'date' in datetime
dataset = dataset.reset_index()
dataset['date'] = pd.to_datetime(dataset['date'])
dataset = dataset.set_index(['user_id', 'date'])

dataset = dataset.sort_index(level=['user_id', 'date'])  # Ordina

current_sequence = []
last_user = None
last_date = None

for (user, date), row in dataset.iterrows():
    if last_user is not None:
        if user == last_user:
            if (date - last_date).days in [1, 2]:
                current_sequence.append(((user, date), row))
                if len(current_sequence) == target_sequence_len:
                    sequences.append(current_sequence.copy())
                    current_sequence.pop(0)
            else:
                current_sequence = [((user, date), row)]
        else:
            current_sequence = [((user, date), row)]
    else:
        current_sequence.append(((user, date), row))

    last_user = user
    last_date = date

print(f"Numero totale di sequenze valide trovate: {len(sequences)}")

Numero totale di sequenze valide trovate: 246049


# **Preparazione dataset finale per NeuralForecast**

In questa fase viene costruito il dataset finale utilizzato per l’addestramento del modello Informer. Per ogni sequenza temporale valida, vengono estratte le principali variabili di interesse: passi, durata del sonno e activity index. Successivamente, per ogni sequenza, viene identificata la data del giorno target, ovvero il giorno successivo alla fine della sequenza, e recuperato il relativo valore di activity_index che rappresenta il valore da prevedere (target y). Solo le sequenze per cui è disponibile il valore target vengono mantenute nel dataset finale.
Infine, i dati vengono strutturati secondo il formato richiesto dalla libreria NeuralForecast, rinominando la colonna dell’utente in unique_id e creando un dataframe pronto per l’addestramento del modello.

In [20]:
final_data = []

for seq in sequences:
    user_id = seq[0][0][0]  # user_id della sequenza
    seq_sorted = sorted(seq, key=lambda x: x[0][1])  # Ordina per data

    # Costruzione del DataFrame della sequenza con anche l'activity_index
    data = []
    for (user, date), row in seq_sorted:
        data.append({
            'user_id': user,
            'date': date,
            'stepsgaitspeed': row['stepsgaitspeed'],
            'sleepduration': row['sleepduration'],
            'activity_index': row['activity_index']
        })
    seq_df = pd.DataFrame(data)

    # Trova la data del giorno target (giorno dopo la fine della sequenza)
    last_date_in_sequence = seq_sorted[-1][0][1]
    target_date = last_date_in_sequence + pd.Timedelta(days=1)

    # Recupero del target (activity_index del giorno successivo)
    try:
        target_row = dataset['activity_index'].loc[(user_id, target_date)]
        target_value = target_row.iloc[0] if isinstance(target_row, pd.Series) else target_row
    except KeyError:
        continue  # Salta la sequenza se non esiste il target

    if pd.isna(target_value):
        continue  # Salta la sequenza se il target è NaN

    seq_df['ds'] = seq_df['date']
    seq_df['y'] = target_value
    final_data.append(seq_df[['user_id', 'ds', 'stepsgaitspeed', 'sleepduration', 'activity_index', 'y']])

# Concatenazione finale
if final_data:
    df_seq = pd.concat(final_data).reset_index(drop=True)
    df_seq.rename(columns={'user_id': 'unique_id'}, inplace=True)
    print("Dataset finale NeuralForecast creato con successo!")
    print(df_seq.head())
else:
    print("Nessuna sequenza completa con target trovata. Dataset vuoto.")


Dataset finale NeuralForecast creato con successo!
                                  unique_id         ds  stepsgaitspeed  \
0  0006769258a59460e68c7d7aa5dce527b6ec7c84 2016-04-25          105.57   
1  0006769258a59460e68c7d7aa5dce527b6ec7c84 2016-04-26           80.80   
2  0006769258a59460e68c7d7aa5dce527b6ec7c84 2016-04-28           97.56   
3  0006769258a59460e68c7d7aa5dce527b6ec7c84 2016-04-29          103.57   
4  0006769258a59460e68c7d7aa5dce527b6ec7c84 2016-04-30          105.68   

   sleepduration  activity_index  y  
0           8.85             7.0  7  
1           8.13             6.0  7  
2           8.60             5.0  7  
3           9.23             7.0  7  
4           8.77             7.0  7  


# **Split Train/Test**

Dopo la preparazione del dataset finale, è stato effettuato uno split dei dati in un set di addestramento e un set di test per valutare le performance del modello in fase di previsione. La suddivisione è avvenuta per utente.

In [21]:
# Controllo che df_seq esista
if 'df_seq' not in locals():
    print("Errore: df_seq non è definito. Esegui prima la preparazione del dataset.")
else:
    from sklearn.model_selection import train_test_split

    # Controllo il dataset
    print("df_seq caricato correttamente!")
    print(df_seq.head())

    # Divido gli utenti unici per evitare leakage
    user_ids = df_seq['unique_id'].unique()
    train_users, test_users = train_test_split(user_ids, test_size=0.3, random_state=42)

    # Costruisco i due dataset
    train_df = df_seq[df_seq['unique_id'].isin(train_users)].reset_index(drop=True)
    test_df = df_seq[df_seq['unique_id'].isin(test_users)].reset_index(drop=True)

    # Stampo le dimensioni
    print(f"Dimensione train: {train_df.shape}")
    print(f"Dimensione test: {test_df.shape}")

df_seq caricato correttamente!
                                  unique_id         ds  stepsgaitspeed  \
0  0006769258a59460e68c7d7aa5dce527b6ec7c84 2016-04-25          105.57   
1  0006769258a59460e68c7d7aa5dce527b6ec7c84 2016-04-26           80.80   
2  0006769258a59460e68c7d7aa5dce527b6ec7c84 2016-04-28           97.56   
3  0006769258a59460e68c7d7aa5dce527b6ec7c84 2016-04-29          103.57   
4  0006769258a59460e68c7d7aa5dce527b6ec7c84 2016-04-30          105.68   

   sleepduration  activity_index  y  
0           8.85             7.0  7  
1           8.13             6.0  7  
2           8.60             5.0  7  
3           9.23             7.0  7  
4           8.77             7.0  7  
Dimensione train: (2249970, 6)
Dimensione test: (969465, 6)


In [22]:
!pip install neuralforecast torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.7/257.7 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.8/285.8 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.0/823.0 kB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.1/68.1 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# **Definizione del modello Informer utilizzando NeuralForcast**

Per la fase di modellazione è stato utilizzato il framework NeuralForecast, che consente di integrare modelli di deep learning per la previsione di serie temporali multivariate. Nello specifico, è stato implementato il modello Informer.
I parametri utilizzati sono stati gestiti direttamente dalla libreria NeuralForecast, che si occupa di costruire e addestrare l’intera architettura encoder-decoder del modello in modo automatico e ottimizzato per il forecasting.

**Il modello Informer - Breve descrizione e funzionamento dell’architettura**

Nel progetto è stato utilizzato Informer, un modello di deep learning basato su architettura Transformer, progettato per la previsione su serie temporali lunghe e multivariate.

A differenza dei Transformer classici, l’Informer è stato ottimizzato per gestire lunghe sequenze riducendo il carico computazionale e migliorando la capacità predittiva grazie a:

- ProbSparse Attention: seleziona solo le informazioni più rilevanti all’interno della sequenza, riducendo il numero di operazioni necessarie e migliorando l’efficienza sui dati lunghi.
- Attention Distillation: riduce progressivamente la complessità della sequenza filtrando i segnali meno importanti e mantenendo solo le componenti dominanti.
- Decoder efficiente (MLP): consente di generare più previsioni in un solo passaggio, velocizzando la fase di inferenza.
- Embedding temporali e posizionali: catturano sia le dipendenze tra i giorni che la posizione temporale delle osservazioni.

**Funzionamento dell’architettura Informer**

L’architettura dell’Informer si basa su due componenti principali:

- Encoder: riceve la sequenza di input (nel nostro caso 15 giorni di dati su attività e sonno) e applica la ProbSparse Attention e la distillazione per estrarre le dipendenze temporali più rilevanti, riducendo la dimensione della sequenza ma mantenendo l’informazione principale.

- Decoder: utilizza l’output dell’encoder e genera la previsione per l’orizzonte desiderato (nel progetto, il giorno 16). Il decoder è progettato per elaborare in modo efficiente anche lunghe finestre temporali e restituire sia la previsione puntuale che le bande di incertezza.

In [23]:
from neuralforecast import NeuralForecast
from neuralforecast.models import Informer
from neuralforecast.losses.pytorch import DistributionLoss

# Numero di giorni della sequenza passata al modello (15 giorni * 3 variabili)
input_size = target_sequence_len * 3  # stepsgaitspeed + sleepduration + activity_index

model = Informer(
    h=1,  # Previsione a 1 giorno avanti
    input_size=input_size,
    hidden_size=512,
    conv_hidden_size=128,
    n_head=8,
    loss=DistributionLoss(distribution='StudentT', level=[80, 95]),
    scaler_type='standard',
    learning_rate=1e-3,
    encoder_layers=2,
    decoder_layers=2,
    max_steps=500,
    val_check_steps=50,
    windows_batch_size=32,
    batch_size=32,
    # Ottimizzazioni
    dropout=0.2,                   # Aiuta contro l'overfitting
    factor=4,                      # Aumenta la sparsity

)

INFO:lightning_fabric.utilities.seed:Seed set to 1


In [24]:
# Istanzio NeuralForecast
nf = NeuralForecast(models=[model], freq='D')

# **Addestramento del modello**

# **Previsioni sul test set**

Modelli:

- Informer ➔ la previsione puntuale del modello
- Informer-hi-80 / hi-95 ➔ parte alta dell'intervallo di predizione (80% - 95%)
- Informer-lo-80 / lo-95 ➔ parte bassa dell'intervallo di predizione
- Informer-median ➔ la mediana della distribuzione predetta